In [1]:
import os
import re
import glob
import datetime
import warnings

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
## This section caches all MODIS MCD43A4 file names in the format listed below, where '?' is a wildcard
try:
    with open('/g/data/oe9/modis_tile_file_list.txt') as f:
        reflectance_file_cache = sorted(l.strip() for l in f.readlines())
except:
    reflectance_file_cache =  sorted(glob.glob(
        '/g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/'
        '????.??.??/MCD43A4.A???????.h??v??.006.*.hdf'
    ))
    with open('/g/data/oe9/modis_tile_file_list.txt', 'w') as f:
        f.writelines(l + '\n' for l in reflectance_file_cache)
len(reflectance_file_cache)

504980

In [3]:
## Rename the bands to something useful
modis_band_map = {
    'Nadir_Reflectance_Band1': 'red_630_690',
    'Nadir_Reflectance_Band2': 'nir1_780_900',
    'Nadir_Reflectance_Band3': 'blue_450_520',
    'Nadir_Reflectance_Band4': 'green_530_610',
    'Nadir_Reflectance_Band5': 'nir2_1230_1250',
    'Nadir_Reflectance_Band6': 'swir1_1550_1750',
    'Nadir_Reflectance_Band7': 'swir2_2090_2350',
}

## define function 'get_reflectance' whose input is tile ID 'h__v__'
## This function searches the cached list for filenames which contain tile 'h__v__
## outputs data as a concatenated dataset with dimensions, time, x ,y 
def get_reflectance(tile):
    assert re.match(r'h\d\dv\d\d', tile), 'tile must be string "h__v__"'
    files = [f for f in reflectance_file_cache if tile in os.path.basename(f)]
    pattern = re.compile(r'MCD43A4.A(?P<year>\d{4})(?P<day>\d{3}).h\d\dv\d\d.006.\d+.hdf')
    dates, parts = [], []
    for f in files:
        assert os.path.isfile(f)
        parts.append(xr.open_dataset(f, chunks=2400,autoclose=True))
        year, day = pattern.match(os.path.basename(f)).groups()
        dates.append(datetime.date(int(year), 1, 1) +
                     datetime.timedelta(days=int(day) - 1))

    dates = pd.to_datetime(dates)
    dates.name = 'time'

    ds = xr.concat(parts, dates)
    out = xr.Dataset()
    for i in map(str, range(1, 8)):
        key = 'Nadir_Reflectance_Band' + i
        data_ok = ds['BRDF_Albedo_Band_Mandatory_Quality_Band' + i] == 0
        out[modis_band_map[key]] = ds[key].where(data_ok).astype('f4')

    out.rename({'YDim:MOD_Grid_BRDF': 'y',
                'XDim:MOD_Grid_BRDF': 'x'}, inplace=True)
    out.time.encoding.update(dict(
        units='days since 1900-01-01', calendar='gregorian', dtype='i4'))
    return out

In [4]:
%%time
ds = get_reflectance('h30v12')
ds

CPU times: user 43.6 s, sys: 43.2 s, total: 1min 26s
Wall time: 4min 30s


In [12]:
dir()

['In',
 'Out',
 '_',
 '_11',
 '_2',
 '_5',
 '_6',
 '_9',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'datetime',
 'ds',
 'exit',
 'f',
 'get_ipython',
 'get_reflectance',
 'glob',
 'modis_band_map',
 'np',
 'os',
 'pd',
 'quit',
 're',
 'reflectance_file_cache',
 'warnings',
 'xr']

In [6]:
ds.x

<xarray.DataArray 'x' (x: 2400)>
array([   0,    1,    2, ..., 2397, 2398, 2399])
Dimensions without coordinates: x

In [ ]:
xr.ds.to_netcdf